# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В моём распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Необходимо построить модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Прежде чем приступать к выполнению заданий, подключу библиотеки.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Всё, что вспомнил. Теперь приступаю к работе

## Откройте и изучите файл

Открываю.

In [ ]:
data = pd.read_csv('/datasets/users_behavior.csv')

Взгляну на то, что внутри. Сразу посмотрю на типы данных в столбцах

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Ну что же, пропусков нет, имена столбцов впорядке. Количество звонков и количество сообщений - вещественные величины. Это невозможно. Позвонить полтора раза или отправить полтора сообщения нельзя. Изменю тип на целочисленный.

In [ ]:
try:
    data=data.astype({ 'calls': 'int', 'messages': 'int'})
    print('Всё получилось')
except:
    print('К сожалению, преобразование не удалось')

Всё получилось


Проверю, что получилось.

In [ ]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0


Теперь всё логично. Звонков и сообщений - целое число. Минут и магабайт - вещественное.

Проверю на дубликаты.

In [ ]:
data.duplicated().sum()

0

Отлично. Дубликатов нет.

**Выводы по пункту 1**
Результаты краткого изучения файла данных показали следующее:
1. Пропусков значений и дубликатов файле не обнаружено.
2. Столбцы количества звонков и количества сообщений имеют вещественный тип данных. Тип данных для указанных столбцов скорректирован.
3. Имена столбцов и значения столбцов вопросов не вызывают.


Движемся дальше.

## Разбейте данные на выборки

Обучающую выборку беру 80%.

In [ ]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=12345)

Создаю переменные для признаков и целевого признака.

In [ ]:
features = data_test.drop('is_ultra', axis=1)
target = data_test['is_ultra']

Валидационную и тестовую, соответственно, по 10%.

In [ ]:
features_valid, features_test, target_valid, target_test = train_test_split(features, target,
                                                                            test_size=0.5, random_state=12345)


Переменные для обучающей выборки.

In [ ]:
features_train = data_train.drop('is_ultra', axis=1)
target_train = data_train['is_ultra']


Получил выборки. Перехожу к исследованию модели.

## Исследуйте модели

Помню об условии:

Построить модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверить accuracy на тестовой выборке самостоятельно.

### Дерево решений

In [ ]:
best_accuracy = 0
best_depth = 0
for depth in range(1,25):
    model = DecisionTreeClassifier(max_depth=depth, random_state=202020)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print("Глубина дерева:", best_depth, "    Качество:", best_accuracy)

Глубина дерева: 6     Качество: 0.7975077881619937


Глубина в 3 узла. Качество более 78%. Проверю другие модели.

### Логистическая регрессия

In [ ]:
model = LogisticRegression(solver='lbfgs')
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Качество:", accuracy)


Качество: 0.7663551401869159


Качество ниже, чем у дерева решений. И не удовлетворяет условию в 75%.

### Случайный лес

In [ ]:
best_accuracy = 0
best_depth = 0
for depth in range(1,15):
    for est in range(2,75):
        for sample in range(2,10):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=12345)
            model.fit(features_train, target_train)
            accuracy = model.score(features_valid, target_valid)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "    Количество деревьев:",
      best_est, "   мин. объектов в узле:", best_sample, "  Качество:", best_accuracy)


Глубина дерева: 8     Количество деревьев: 6    мин. объектов в узле: 5   Качество: 0.822429906542056


**Краткий вывод**

Таким образом, наивысшим качетством обладает модель Случайного леса - более 81% процента точности.

## Проверьте модель на тестовой выборке

На тестовой выборке проверяю лучшую модель из предыдущего пункта - модель Случайного леса.

Значения гиперпараметров беру из расчёта в предыдущем пункте

In [ ]:
model = RandomForestClassifier(max_depth=6, n_estimators=34, min_samples_leaf=2, random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.8012422360248447


**Вывод**

Точность на тестовой выборке близка к валидационной, но не превышает её. Значит переобучения модели нет.

## (бонус) Проверьте модели на адекватность

Честно говоря, не сообразил, как лучше проверить. Сделаю так.
Встречал, что можно применить DummyClassifier, но не совсем разобрался с этим.

In [ ]:
(data['is_ultra']==0).sum() / data.shape[0]

0.693528313627878

У моделей точность выше. Признаю их адекватными.

## Вывод

В процессе выполнения проекта обнаружено следующее:
1. Пропусков значений и дубликатов файле не обнаружено;
2. Столбцы количества звонков и количества сообщений имеют вещественный тип данных. Тип данных для указанных столбцов скорректирован;
3. Имена столбцов и значения столбцов вопросов не вызывают;
4. Созданы тестовая, обучающая и валидационная выборки;
5. Исследование модели показало, что самым высоким качеством обладает модель Случайного леса;
6. Проверка модели на тестовой выборке говорит об отсутствии проблем с переобучением модели;
7. Проверка модели на адекватность подтвердила адекватность моих моделей.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
